# Developing Word Embeddings

Rather than use pre-trained embeddings (as we did in the baseline_deep_dive notebook), we can train word embeddings using our own dataset. In this notebook, we demonstrate the training process for producing word embeddings using the word2vec, GloVe, and fastText models. We'll utilize the STS Benchmark dataset for this task. 

# Table of Contents
* [Data Loading and Preprocessing](#Load-and-Preprocess-Data)
* [Word2Vec](#Word2Vec)
* [fastText](#fastText)
* [GloVe](#GloVe)
* [Concluding Remarks](#Concluding-Remarks)

In [1]:
import gensim
import sys

## Load and Preprocess Data

In [2]:
sys.path.append("../../../")  ## set the environment path

BASE_DATA_PATH = "../../../data"

from utils_nlp.dataset.stsbenchmark import STSBenchmark
from utils_nlp.dataset.preprocess import (
    to_lowercase,
    to_spacy_tokens,
    rm_spacy_stopwords,
)

In [3]:
# Initializing this instance runs the downloader and extractor behind the scenes, then convert to dataframe
stsTrain = STSBenchmark("train", base_data_path=BASE_DATA_PATH).as_dataframe()

In [4]:
# train preprocessing
df_low = to_lowercase(stsTrain)  # convert all text to lowercase
sts_tokenize = to_spacy_tokens(df_low)  # tokenize normally
sts_train = rm_spacy_stopwords(sts_tokenize)  # tokenize with removal of stopwords

In [5]:
#append together the sentence1_tokens and sentence2_tokens columns to get a list of all tokenized sentences
sentences = sts_train["sentence1_tokens"].append(sts_train["sentence2_tokens"])

In [6]:
sentences[:10]

0                       [a, plane, is, taking, off, .]
1            [a, man, is, playing, a, large, flute, .]
2    [a, man, is, spreading, shreded, cheese, on, a...
3                 [three, men, are, playing, chess, .]
4                 [a, man, is, playing, the, cello, .]
5                        [some, men, are, fighting, .]
6                             [a, man, is, smoking, .]
7               [the, man, is, playing, the, piano, .]
8    [a, man, is, playing, on, a, guitar, and, sing...
9    [a, person, is, throwing, a, cat, on, to, the,...
dtype: object

## Word2Vec

Word2vec is a predictive model for learning word embeddings from text. Word embeddings are learned such that words that share common contexts in the corpus will be close together in the vector space. There are two different model architectures that can be used to produce word2vec embeddings: continuous bag-of-words (CBOW) or continuous skip-gram. The former uses a window of surrounding words (the "context") to predict the current word and the latter uses the current word to predict the surrounding context words. See this [tutorial](https://www.guru99.com/word-embedding-word2vec.html#3) on word2vec for more detailed background on the model.

The gensim Word2Vec model has many different parameters (see [here](https://radimrehurek.com/gensim/models/word2vec.html#gensim.models.word2vec.Word2Vec)) but the ones that are useful to know about are:  
- size: length of the word embedding/vector (defaults to 100)
- window: maximum distance between word being predicted and the current word (defaults to 5)
- min_count: ignores all words that have a frequency lower than this value (defaults to 5)
- workers: number of worker threads used to train the model (defaults to 3)
- sg: training algorithm; 1 for skip-gram and 0 for CBOW (defaults to 0)

In [7]:
from gensim.models import Word2Vec

word2vec_model = Word2Vec(sentences, size=100, window=5, min_count=5, workers=3, sg=0)

Now that the model is trained we can:

1. Query for the word embeddings of a given word. 
2. Inspect the model vocabulary
3. Save the word embeddings

In [8]:
# 1. Let's see the word embedding for "apple" by accessing the "wv" attribute and passing in "apple" as the key.
print("Embedding for apple:", word2vec_model.wv["apple"])

# 2. Inspect the model vocabulary by accessing keys of the "wv.vocab" attribute. We'll print the first 20 words
print("\nFirst 30 vocabulary words:", list(word2vec_model.wv.vocab)[:20])

# 3. Save the word embeddings. We can save as binary format (to save space) or ASCII format
word2vec_model.wv.save_word2vec_format("word2vec_model", binary=True)  # binary format
word2vec_model.wv.save_word2vec_format("word2vec_model", binary=False)  # ASCII format

Embedding for apple: [-1.30064473e-01  1.84295833e-01 -1.53965428e-01 -9.69498605e-02
  4.99420874e-02 -1.23197936e-01  7.28140250e-02 -4.12699208e-02
  2.47626036e-01 -2.69805547e-04 -7.65557750e-04  2.08947986e-01
  7.81186996e-03  5.42742060e-03  5.25087006e-02  2.47807354e-01
 -2.48165410e-02  9.91394650e-03  3.54040265e-02 -2.14830145e-01
  2.24868301e-02  1.52286962e-01  1.85761824e-01  2.33249858e-01
 -1.46878466e-01 -7.60829672e-02  4.50950442e-03  1.15145534e-01
 -9.11297649e-02  6.20169528e-02 -5.24968617e-02 -8.68254527e-02
 -1.77496113e-04  8.58828798e-02  1.19839951e-01  2.51445977e-04
 -3.06774918e-02  2.70280894e-03 -9.14655998e-02  5.54770082e-02
  6.70319721e-02 -1.10063367e-01 -9.94274616e-02 -1.62537303e-02
  1.07709818e-01 -1.17890313e-01 -1.68436840e-02  2.67276943e-01
  1.66485235e-02 -1.05556019e-01  8.72049183e-02 -2.79379219e-01
 -7.61673898e-02 -1.26047105e-01 -2.10570037e-01  1.06335968e-01
 -1.13933079e-01  8.91806409e-02  2.40348503e-02  1.27991261e-02
 -9.

## fastText

fastText is an unsupervised algorithm created by Facebook Research for efficiently learning word embeddings. fastText is significantly different than word2vec or GloVe in that these two algorithms treat each word as the smallest possible unit to find an embedding for. Conversely, fastText assumes that words are formed by an n-gram of characters (i.e. 2-grams of the word "language" would be {la, an, ng, gu, ua, ag, ge}). The embedding for a word is then composed of the sum of these character n-grams. This has advantages when finding word embeddings for rare words and words not present in the dictionary, as these words can still be broken down into character n-grams. Typically, for smaller datasets, fastText performs better than word2vec or GloVe. See this [tutorial](https://fasttext.cc/docs/en/unsupervised-tutorial.html) on fastText for more detail.

The gensim fastText model has many different parameters (see [here](https://radimrehurek.com/gensim/models/fasttext.html#gensim.models.fasttext.FastText)) but the ones that are useful to know about are:  
- size: length of the word embedding/vector (defaults to 100)
- window: maximum distance between word being predicted and the current word (defaults to 5)
- min_count: ignores all words that have a frequency lower than this value (defaults to 5)
- workers: number of worker threads used to train the model (defaults to 3)
- sg: training algorithm- 1 for skip-gram and 0 for CBOW (defaults to 0)
- iter: number of epochs (defaults to 5)


In [9]:
from gensim.models.fasttext import FastText

fastText_model = FastText(size=100, window=5, min_count=5, sentences=sentences, iter=5)

We can utilize the same attributes as we saw above for word2vec due to them both originating from the gensim package

In [10]:
# 1. Let's see the word embedding for "apple" by accessing the "wv" attribute and passing in "apple" as the key.
print("Embedding for apple:", fastText_model.wv["apple"])

# 2. Inspect the model vocabulary by accessing keys of the "wv.vocab" attribute. We'll print the first 20 words
print("\nFirst 30 vocabulary words:", list(fastText_model.wv.vocab)[:20])

# 3. Save the word embeddings. We can save as binary format (to save space) or ASCII format
fastText_model.wv.save_word2vec_format("fastText_model", binary=True)  # binary format
fastText_model.wv.save_word2vec_format("fastText_model", binary=False)  # ASCII format

Embedding for apple: [-0.19466035  0.02329457  0.11905755  0.43202105  0.29234868 -0.4173747
 -0.42871934 -0.587514   -0.24620762 -0.30886024 -0.04068367  0.20132142
 -0.1593995  -0.34693947 -0.05454068  0.21118519  0.20061074  0.33920124
  0.13465068 -0.16492505 -0.01792471  0.3517471  -0.42507643 -0.14185262
  0.6766511  -0.35682997  0.38852996  0.08338872 -0.16927068  0.00101932
  0.01033709 -0.00513317 -0.15251048 -0.07668231  0.02508747 -0.16725563
  0.13578647  0.5188022   0.4219404  -0.29186445 -0.35036987  0.04769979
 -0.23967543 -0.03550959 -0.4072291   0.4920213   0.30146047 -0.569966
  0.12033249 -0.24960376 -0.20398642 -0.37427858  0.04139522  0.28986236
 -0.31172943  0.7363574  -0.43040937  0.24302956 -0.2891899  -0.12707426
 -0.26763597 -0.3471016   0.08912586 -0.20722611  0.1529707   0.39230242
 -0.23503402 -0.00332095 -0.04347242 -0.00989339  0.08801552 -0.36916256
 -0.13720557  0.40390077 -0.21936806 -0.10426865 -0.18858872  0.15547332
 -0.3519439   0.00505178  0.10296

## GloVe

GloVe is an unsupervised algorithm for obtaining word embeddings created by the Stanford NLP group. Training occurs on word-word co-occurrence statistics with the objective of learning word embeddings such that the dot product of two words' embeddings is equal to the words' probability of co-occurrence. See this [tutorial](https://nlp.stanford.edu/projects/glove/) on GloVe for more detailed background on the model. 

Gensim doesn't have an implementation of the GloVe model, so we suggest getting the code directly from Stanford NLP, who developed the algorithm (https://github.com/stanfordnlp/GloVe). Run the following commands to clone the repo and then make. Clone the repo in the same location as this notebook! Otherwise, the paths below will need to be modified.  

    git clone http://github.com/stanfordnlp/glove    
    cd glove && make  

### Train GloVe vectors

Training GloVe embeddings requires some data prep and then 4 steps (also documented in the original Stanford NLP repo [here](https://github.com/stanfordnlp/GloVe/tree/master/src)).

**Step 0: Prepare Data**
   
In order to train our GloVe vectors, we first need to save our corpus as a text file with all words separated by 1+ spaces or tabs. Each document/sentence is separated by a new line character.

In [11]:
#save our corpus as tokens delimited by spaces with new line characters in between sentences
with open('sentences.txt', 'w', encoding='utf8') as file:
    for sent in sentences:
        file.write(" ".join(sent) + "\n")

**Step 1: Build Vocabulary**

Run the vocab_count executable. There are 3 optional parameters:
1. min-count: lower limit on how many times a word must appear in dataset. Otherwise the word is discarded from vocabulary.
2. max-vocab: upper bound on the number of vocabulary words to keep
3. verbose: 0, 1, or 2 (default)

Then provide the path to the text file we created in Step 0 (<"sentences.txt">) followed by a file path that we'll save the vocabulary to ("glove/build/vocab.txt")

In [12]:
!"glove/build/vocab_count" -min-count 5 -verbose 2 <"sentences.txt"> "glove/build/vocab.txt"

BUILDING VOCABULARY
Processed 0 tokens.100000 tokens.Processed 129989 tokens.
Counted 11986 unique words.
Truncating vocabulary at min count 5.
Using vocabulary of size 3166.



**Step 2: Construct Word Co-occurrence Statistics**

Run the cooccur executable. There are many optional parameters, but we list the top ones here:
1. symmetric: 0 for only looking at left context, 1 (default) for looking at both left and right context
2. window-size: number of context words to use (default 15)
3. verbose: 0, 1, or 2 (default)
4. vocab-file: path/name of the vocabulary file created in Step 1
5. memory: soft limit for memory consumption, default 4
6. max-product: limit the size of dense co-occurrence array by specifying the max product (integer) of the frequency counts of the two co-occurring words

Then provide the path to the text file we created in Step 0 (<"sentences.txt">) followed by a file path that we'll save the co-occurrences to ("glove/build/cooccurrence.bin")

In [13]:
!"glove/build/cooccur" -memory 4 -vocab-file "glove/build/vocab.txt" -verbose 2 -window-size 15 <"sentences.txt"> "glove/build/cooccurrence.bin"

COUNTING COOCCURRENCES
window size: 15
context: symmetric
max product: 13752509
overflow length: 38028356
Reading vocab from file "glove/build/vocab.txt"...loaded 3166 words.
Building lookup table...table contains 10023557 elements.
Processing token: 0100000Processed 129989 tokens.
Writing cooccurrences to disk.......2 files in total.
Merging cooccurrence files: processed 0 lines.0 lines.100000 lines.200000 lines.300000 lines.Merging cooccurrence files: processed 340325 lines.



**Step 3: Shuffle the Co-occurrences**

Run the shuffle executable. The parameters are as follows:
1. verbose: 0, 1, or 2 (default)
2. memory: soft limit for memory consumption, default 4
3. array-size: limit to the length of the buffer which stores chunks of data to shuffle before writing to disk

Then provide the path to the co-occurrence file we created in Step 2 (<"glove/build/cooccurrence.bin">) followed by a file path that we'll save the shuffled co-occurrences to ("glove/build/cooccurrence.shuf.bin")

In [14]:
!"glove/build/shuffle" -memory 4 -verbose 2 <"glove/build/cooccurrence.bin"> "glove/build/cooccurrence.shuf.bin"

SHUFFLING COOCCURRENCES
array size: 255013683
Shuffling by chunks: processed 0 lines.processed 340325 lines.
Wrote 1 temporary file(s).
Merging temp files: processed 0 lines.340325 lines.Merging temp files: processed 340325 lines.



**Step 4: Train GloVe model**

Run the glove executable. There are many parameter options, but the top ones are listed below:
1. verbose: 0, 1, or 2 (default)
2. vector-size: dimension of word embeddings (50 is default)
3. threads: number threads, default 8
4. iter: number of iterations, default 25
5. eta: learning rate, default 0.05
6. binary: whether to save binary format (0: text = default, 1: binary, 2: both)
7. x-max: cutoff for weighting function, default is 100
8. vocab-file: file containing vocabulary as produced in Step 1
9. save-file: filename to save vectors to 
10. input-file: filename with co-occurrences as returned from Step 3

In [15]:
!"glove/build/glove" -save-file "glove/build/GloVe_vectors" -threads 8 -input-file \
"glove/build/cooccurrence.shuf.bin" -x-max 10 -iter 15 -vector-size 50 -binary 2 \
-vocab-file "glove/build/vocab.txt" -verbose 2

TRAINING MODEL
Read 340325 lines.
Initializing parameters...done.
vector size: 50
vocab size: 3166
x_max: 10.000000
alpha: 0.750000
04/29/19 - 01:26.33PM, iter: 001, cost: 0.098453
04/29/19 - 01:26.33PM, iter: 002, cost: 0.084751
04/29/19 - 01:26.33PM, iter: 003, cost: 0.074604
04/29/19 - 01:26.33PM, iter: 004, cost: 0.071038
04/29/19 - 01:26.33PM, iter: 005, cost: 0.067709
04/29/19 - 01:26.33PM, iter: 006, cost: 0.064181
04/29/19 - 01:26.33PM, iter: 007, cost: 0.059996
04/29/19 - 01:26.33PM, iter: 008, cost: 0.055268
04/29/19 - 01:26.33PM, iter: 009, cost: 0.050708
04/29/19 - 01:26.33PM, iter: 010, cost: 0.046754
04/29/19 - 01:26.33PM, iter: 011, cost: 0.043402
04/29/19 - 01:26.33PM, iter: 012, cost: 0.040575
04/29/19 - 01:26.33PM, iter: 013, cost: 0.038056
04/29/19 - 01:26.33PM, iter: 014, cost: 0.035843
04/29/19 - 01:26.33PM, iter: 015, cost: 0.033807


### Inspect Word Vectors

Like we did above for the word2vec and fastText models, let's now inspect our word embeddings

In [16]:
#load in the saved word vectors
glove_wv = {}
with open("glove/build/GloVe_vectors.txt", encoding='utf-8') as f:
    for line in f:
        split_line = line.split(" ")
        glove_wv[split_line[0]] = [float(i) for i in split_line[1:]]

In [17]:
# 1. Let's see the word embedding for "apple" by passing in "apple" as the key.
print("Embedding for apple:", glove_wv["apple"])

# 2. Inspect the model vocabulary by accessing keys of the "wv.vocab" attribute. We'll print the first 20 words
print("\nFirst 30 vocabulary words:", list(glove_wv.keys())[:20])

Embedding for apple: [-0.015303, -0.0512, -0.011988, 0.429914, 0.246523, 0.009762, 0.153154, -0.178636, 0.061723, 0.108515, -0.166807, -0.033258, -0.046394, 0.081953, -0.209458, 0.194758, 0.179153, 0.23262, -0.118717, -0.053151, -0.018892, -0.037714, -0.067396, 0.057499, 0.179459, 0.004552, -0.203058, 0.243629, -0.294976, 0.123971, 0.368613, 0.190665, -0.16738, -0.0599, 0.119195, -0.030108, -0.254778, -0.007862, -0.036998, 0.060919, -0.210459, 0.293917, 0.045603, -0.01104, 0.075651, -0.120635, -0.133497, -0.372606, -0.152981, 0.009014]

First 30 vocabulary words: ['.', 'a', 'the', 'in', ',', 'is', 'to', 'of', 'and', 'on', 'man', '-', "'s", 'with', 'for', 'at', 'woman', 'are', 'that', 'two']


# Concluding Remarks

In this notebook we have shown how to train word2vec, GloVe, and fastText word embeddings on the STS Benchmark dataset. FastText is typically regarded as the best baseline for word embeddings (see [blog](https://medium.com/huggingface/universal-word-sentence-embeddings-ce48ddc8fc3a)) and is a good place to start when generating word embeddings. Now that we generated word embeddings on our dataset, we could also repeat the baseline-deep-dive notebook using these embeddings (versus the pre-trained ones from the internet). 